![banner](Banner.jpg)

# Métodos de Monte Carlo

En este ejercicio vamos a implementar una  solución para los problemas de aprendizaje por refuerzo basada en los métodos de Monte Carlo. 

Recuerde que el método de Monte Carlo consiste en la colección de muestras calculando los valores para la secuencia completa de los estados hasta el estado final del episodio. 
Una vez se han coleccionado "suficientes" muestras, el valor de los estados se toma como el valor promedio de las muestras sobre las cuales apareció el estado.

## Ambiente

En este laboratorio vamos a utilizar el método de Monte Carlo para construir un agente que resuleva el ambiente de la cruz que utilizamos en los videos.

<img src="cruz.png" width="200"/>

Como se muestra en la figura, el ambiente tiene cinco estados (`A` a `E`) configurados en una cruz. El ambiente tiene dos estados finales (`A` y `D`) y cuatro acciones posibles (`up`, `right`, `down`, `left`) para los estados `B`,`C`,`E` y una acción posible (`exit`) para los estados `A` y `D`. La codificación de las acciones para cada uno de los estados y su acción correspondiente se muestra en la tabla a continuación. La primera posición es la probabilidad, la segunda el estado de llegada y la tercera la recompensa.

![Encoding_table](table.png)



#### Definición del ambiente

El ambiente se define dentro de la clase `MDP`, que recibe como parámetro el tablero con la codificación del MDP, las dimensiones del tablero (que se almacenan en los astributos `nrows` y `ncols`) y el estado inicial (que se almacena en el atributo `initial_state`).
Los valores codificados en el tablero se almacenarán en dos atributos de la clase, un atributo `transitions` y otro `rewards`. El atributo `transitions` es un mapa (diccionario) con llaves de tipo `(estado, accion)`, cuyo valor para cada llave es la lista de `(probability, state)` posibles desde ese estado después de tomar esa acción. Por ejemplo, en la llave `('B','left')` debería existir una lista cuyo único elemento es `(1, 'B')`. El atributo `rewards` funciona de manera similar. Es un mapa donde para cada llave `(estado, accion)`, se tiene como valor una lista de los posibles `(reward, state)` que se pueden conseguir desde ese estado tomando esa acción. Por ejemplo, en la llave `('B','up')` debería existir una lista que tenga como elementos `(0, 'B')` y `(0, 'C')`. Las transiciones y recompensas que no existan se deben guardar como None, no como una lista vacía. 

La forma de representar acciones es como un string del siguiente conjunto `["up", "right", "down", "left", "exit"]`. La forma de representar estados también es como un string parte del conjunto `['B', 'C', 'E', 'A', 'D']`.

Adicionalmente la clase debe tener los atributos `initial_state` y `state` que corresponden al estado inicial, dado por parámetro, y el estado actual del agente en el ambiente, inicializado en el estado inicial.

Note también que los estados `A` y `D` tienen una única acción asociada que lleva a ningun estado, marcandolos como estados finales.

In [1]:
#Librerias requeridas
import numpy as np
import random
import sys # Usado para manejar errores

#Definición de la clase principal
class MDP:
    def __init__(self, table: list[list[tuple[float,str,int]]], dimensions:tuple[int, int], initial_state:str):
        self.nrows = dimensions[0]
        self.ncols = dimensions[1]
        self.initial_state = initial_state
        self.state = initial_state
        
        self.transitions = {}
        self.rewards = {}
        
        # Mapeo de índices a nombres de estados y acciones
        states_map = ['B', 'C', 'E', 'A', 'D']
        actions_map = ['up', 'right', 'down', 'left', 'exit']
        
        for i in range(len(states_map)):
            for j in range(len(actions_map)):
                current_state = states_map[i]
                current_action = actions_map[j]
                entry = table[i][j]
                
                key = (current_state, current_action)
                
                if entry is None:
                    self.transitions[key] = None
                    self.rewards[key] = None
                else:
                    t_list = []
                    r_list = []
                    for (prob, next_s, rew) in entry:
                        t_list.append((prob, next_s))
                        r_list.append((rew, next_s)) 
                    
                    self.transitions[key] = t_list
                    self.rewards[key] = r_list
    
    def get_action_index(self, action:str) -> int:
        actions = ['up', 'right', 'down', 'left']
        index = 0
        for a in actions:
            if action == a:
                return index
            index += 1
        return -1

    def get_states(self) -> list[str]:
        states = set(())
        for t in self.transitions:
            states.add(t[0])
        return list(states)

    def get_possible_states(self, state:str, action:str) -> tuple[float,int,list[tuple[int,int]]]:
        probability = []
        rewards=[]
        states=[]
        

        if (state, action) not in self.transitions or self.transitions[(state, action)] is None:
             raise Exception(f"Estado o acción inválida: {(state, action)}")

        for i in range(len(self.transitions[(state,action)])):
            rewards.append(self.rewards[(state,action)][i][0])
            probability.append(self.transitions[(state,action)][i][0])
            states.append(self.transitions[(state,action)][i][1])
        return probability, rewards, states
    
    def get_current_state(self) -> str:

        return self.state
    
    def get_possible_actions(self, state:str) -> list[str]:

        possible_actions = []
        all_actions = ['up', 'right', 'down', 'left', 'exit']
        
        if state is None:
            return []
            
        for action in all_actions:
            if self.transitions.get((state, action)) is not None:
                possible_actions.append(action)
        return possible_actions
    
    def do_action(self, action:str) -> tuple[float, str]:
        if action not in self.get_possible_actions(self.state):
            raise Exception(f"Acción '{action}' no es posible desde el estado '{self.state}'")
            
        t_list = self.transitions[(self.state, action)]
        r_list = self.rewards[(self.state, action)]
        
        probs = [t[0] for t in t_list]
        next_states = [t[1] for t in t_list]
        rewards = [r[0] for r in r_list]
        indices = list(range(len(probs)))
        chosen_index = random.choices(indices, weights=probs, k=1)[0]
        
        chosen_next_state = next_states[chosen_index]
        chosen_reward = rewards[chosen_index]
        self.state = chosen_next_state
        return (chosen_reward, chosen_next_state)

    def reset(self):
        self.state = self.initial_state
    
    def is_terminal(self, state:str) -> bool:
        return state == 'A' or state == 'D'

In [2]:
#Definición del ambiente

table = [[[(0.8,'B',0), (0.2,'C',0)], [(0.7,'C',0), (0.3,'B',0)],[(0.7,'C',0), (0.3,'B',0)],[(1,'B',0)],None],
         [[(0.7,'A',0), (0.1,'D',0), (0.2,'B',0)],[(0.7,'D',0), (0.1,'E',0), (0.2,'A',0)],[(0.7,'E',0), (0.1,'B',0), (0.2,'D',0)],[(0.7,'C',0), (0.1,'A',0), (0.2,'E',0)],None],
         [[(0.7,'C',0), (0.3,'E',0)],[(0.8,'E',0),(0.2,'C',0)],[(1,'E',0)],[(0.9,'E',0), (0.1,'C',0)], None],
         [None, None, None, None, [(1,None,-10)]],
         [None, None, None, None, [(1,None,10)]]]

In [3]:
#Pruebas estructura del ambiente.

env = MDP(table, (5,5), 'B')

try:
    env.nrows
    assert type(env.nrows) is int, "Las dimensiones del ambiente deben ser enteras"
except:
    raise Exception("El atributo nrows no está definido")
try:
    env.ncols
    assert type(env.ncols) is int, "Las dimensiones del ambiente deben ser enteras"
except:
    raise Exception("El atributo ncols no está definido")
try:
    env.initial_state
    assert type(env.initial_state) is str, "El estado inicial debe corresponder al nombre de uno de los estados, un string"
except:
    raise Exception("El atributo initial_state no está definido")
try:
    env.state
    assert type(env.state) is str, "el estado del ambiente deben ser un string"
except:
    raise Exception("El atributo state no está definido")
    


In [4]:
#Pruebas estructura del ambiente parte 2.

env = MDP(table, (5,5), 'B')


In [5]:
#Pruebas estructura del ambiente parte 3. 

env = MDP(table, (5,5), 'C')

In [6]:
#Pruebas adicionales


#### Acciones del agente

Defina la función `do_action` que ejecuta la acción tomada por el agente dentro del ambiente. Esta función recibe como parámetro el nombre de la acción a ejecutar y retorna una tupla con el valor de la recompensa del estado de llegada de la acción y el estado de llegada. 

Las acciones posibles del agente son `up`, `right`, `down` y `left` para los estados `B`, `C`, `E` y `exit` para los estados `A` y `D` siguiendo la función de transición dada por el MDP. El método debería lanzar una excepción cuando se intenta tomar una acción que no es posible. Por ejemplo, si intenta hacer `exit` desde el estado `B`. 

In [7]:
#Pruebas de ejecución de las acciones

env = MDP(table, (5,5), 'B')

try:
    env.do_action
except:
    raise Exception("La función do_action no está definida")


In [8]:
#Pruebas de ejecución de las acciones parte 2

env = MDP(table, (5,5), 'B')


In [9]:
#Pruebas de ejecución de las acciones parte 3

env = MDP(table, (5,5), 'B')


In [10]:
#PRuebas adicionales



#### Estado actual

Defina la función `get_current_state` que retorna el nombre del estado actual del agente. Esta función no recibe ningún parámetro.

In [11]:
#Pruebas estado actual

env = MDP(table, (5,5), 'B')

try:
    env.get_current_state
    assert type(env.get_current_state()) is str, "La función debe retornar el nombre del estado"
except:
    raise Exception("La función do_action no está definida")


In [12]:
#Pruebas adicionales



#### Obtener las acciones

Defina la función `get_possible_actions` que recibe el estado actual del agente por parámetro y retorna una lista de las acciones válidas para el estado dado.

In [13]:
#Pruebas de acciones posibles

env = MDP(table, (5,5), 'B')

try:
    env.get_possible_actions
    assert type(env.get_possible_actions('B')) is list, "la función get_possible_actions debe ser una lista con las acciones disponibles para el estado"
except:
    raise Exception("La función get_possible_actions no está definida")


In [14]:
#Pruebas de acciones posibles parte 2

env = MDP(table, (5,5), 'B')


In [15]:
#Pruebas adicionales




#### Reinicializar el ambiente

Defina la función `reset` que no recibe parámetros ni retorna ningún valor. El efecto de esta función es restablecer el ambiente a su estado inicial.



In [16]:
#Pruebas de reset

env = MDP(table, (5,5), 'B')

try:
    env.reset
except:
    raise Exception("La función reset no está definida")

# BEGIN HIDDENT TESTS
env = MDP(table, (5,5), 'B')

env.state = 'E'
env.reset()
assert env.get_current_state() == env.initial_state, "No se esta retornando al estado inicial. {env.get_current_state()} y no {env.initial_state}"

In [17]:
#Pruebas adicionales




#### Estados terminales

Defina la función `is_terminal` que recibe un estado com parámetro y determina si es un estado final o no. En nuestro caso los estados finales serán los estados `A` y `D`
La función debe retornar un booleano que corresponde a si el estado pasado por parámetro es terminal o no.

In [18]:
#Pruebas terminación

env = MDP(table, (5,5), 'B')

try:
    env.is_terminal
    assert type(env.is_terminal('B')) is bool, "La función debe retornar un booleano"
except:
    print("La función is_terminal no está definida")


In [19]:
#Pruebas adicionales


## Método de Monte Carlo

Ahora construiremos un agente que utilice el método de Monte Carlo para resolver el MDP definido.

El comportamiento del agente (de Monte Carlo) esta dado por dos momentos. El proceso de recolección de muestras y el proceso de explotación de las mismas, es decir, el cálculo de la política del agente. Usted debe implementar el comportamiento del agente dado que, ejecutando episodios como muestras, sea capaz de calcular los valores para los estados.

#### 1. Creación del agente

Implemente la classe `MCM` para solucionar el ambiente de la cruz. La clase debe tener los siguientes atributos:
- El ambiente `env`, que se recibe por parámetro. La única interacción que se tendrá con el ambiente es para ejecutar acciones y recibir la retroalimentación correspondiente 
- El factor de descuento `discount`, que se pasa por parámetro. Si no se da este parámetro el valor por defecto es `0.9` 
- La cantidad de iteraciones `episodes` a ejecutar, que se pasa por parámetro. Si no se da este parámetro el valor por defecto es 1000
- El threshold `threshold` de ejecución que pasa por parámetro. Determina cuándo dos números son lo suficientemente similares para afirmar que convergieron. Si no se pasa este parámetro, su valor por defecto de `0.00001`. 
- Los valores de los estados `values` que se definen como un mapa de parejas `{estado:valor}` almacenando los valores para cada uno de los estados. Como en casos anteriores inicializaremos este atributo como un mapa vacio (i.e., `{}`) que se llenará de acuerdo a los valores observados. 
- Las recompensas `rewards` que almacena el valor obtenido de la recompensa de ejecutar una acción para cada estado. El mapa de `rewards` como un mapa `{(estado,acción): valor}`. Al igual que en el caso anterior, este mapa se inicializa vácio (i.e., `{}`) y se supone que los valores iniciales son todos `0` si no se han definido antes de usarlos. 
- La política `policy`, que que lleva la política de las mejores acciones a ejecutar por el agente. Este atributo se define como un mapa de parejas `{estado:acción}`, inicializandolo vació. 
- La cantidad de muestras `samples` ejecutadas por el agente, inicializada en 0
- La cantidad de episodios de exploración `explore` a ejecutar por el agente, inicializado en 0

In [22]:
#Librerias a importar
import numpy as np
import random

class MCM():

    def __init__(self, environment:MDP, discount:float = 0.9, iterations:int =1000, threshold:float =0.00001):
        self.env = environment
        self.discount = discount
        self.episodes = iterations
        self.threshold = threshold
        self.values = {} 
        self.rewards = {} 
        self.policy = {}
        self.samples = 0
    
    def get_values(self, state:str) -> int:
        if self.values.get(state) == None:
            return 0
        return self.values.get(state)
    
    def generate_episode(self) -> list[tuple[str,str,int]]:
    
        self.env.reset() 
        episode_history = []
        
        while self.env.get_current_state() is not None:
            current_state = self.env.get_current_state()
            
            possible_actions = self.env.get_possible_actions(current_state)
            
            if not possible_actions:
                break
                
            chosen_action = random.choice(possible_actions)
            (reward, next_state) = self.env.do_action(chosen_action)
            episode_history.append((current_state, chosen_action, reward))
            self.samples += 1
            
        return episode_history

    def check_convergence(self, current_values:dict) -> bool:     
        is_converged = True
        all_states = set(self.values.keys()) | set(current_values.keys())

        for state in all_states:
            old_val = self.values.get(state, 0) 
            new_val = current_values.get(state, 0)
            
            if abs(new_val - old_val) > self.threshold:
                is_converged = False
                break
        
        if not is_converged:
           
            self.values = current_values.copy() 
            return False
        return True

    def calculate_rewards(self, episode:list[tuple[str,str,int]]) -> dict[list[int]]:
        
        returns_per_state = {}
        G = 0 
        visited_states = set()
        for (state, action, reward) in reversed(episode):
            G = reward + self.discount * G
            
            if state not in visited_states:
                if state not in returns_per_state:
                    returns_per_state[state] = []
                returns_per_state[state].append(G)
                visited_states.add(state)
                
        return returns_per_state
        
    def run(self) -> tuple[dict, dict]:
        # Ejecuta el método de Monte Carlo por el número de episodios especificado.
        values = {}
        for i in range(self.episodes):
            episode = self.generate_episode()
            rewards = self.calculate_rewards(episode)
            
            # Actualiza las recompensas con las recompensas de cada estado
            for state in rewards:
                if state not in self.rewards:
                    self.rewards[state] = []
                self.rewards[state].extend(rewards[state])
            
            for state in rewards:
                # El valor del estado esta dado por la media de las recompensas obtenidas para el estado durante el episodio
                values[state] = np.mean(self.rewards[state])
            if self.check_convergence(values):
                print("Convergio en el episodio",i)
                break
        
        # La política para cada estado usando los valores actualizados
        for state in self.env.get_states():  
            possible_actions = self.env.get_possible_actions(state)
            if len(possible_actions) > 0:
                values = []
                for action in possible_actions:
                    prob, reward, next_state = self.env.get_possible_states(state, action)
                    value = 0
                    for i in range(len(prob)):
                        previous_val = self.get_values(next_state[i]) if next_state[i] != None else 0
                        value += prob[i]*(reward[i] + self.discount * previous_val)
                    values.append(value)
                best_action = possible_actions[np.argmax(values)]
                self.policy[state] = best_action
        
        # Returna los valores calculados y la política
        return self.values, self.policy

In [23]:
#Escenarios de prueba

table = [[[(0.8,'B',0), (0.2,'C',0)], [(0.7,'C',0), (0.3,'B',0)],[(0.7,'C',0), (0.3,'B',0)],[(1,'B',0)],None],
         [[(0.7,'A',0), (0.1,'D',0), (0.2,'B',0)],[(0.7,'D',0), (0.1,'E',0), (0.2,'A',0)],[(0.7,'E',0), (0.1,'B',0), (0.2,'D',0)],[(0.7,'C',0), (0.1,'A',0), (0.2,'E',0)],None],
         [[(0.7,'C',0), (0.3,'E',0)],[(0.8,'E',0),(0.2,'C',0)],[(1,'E',0)],[(0.9,'E',0), (0.1,'C',0)], None],
         [None, None, None, None, [(1,None,-10)]],
         [None, None, None, None, [(1,None,10)]]]

environment = MDP(table, (5,5), 'B')

In [24]:
mcm = MCM(environment, discount=0.9, iterations=1000, threshold=0.00001)

try:
    mcm.env
    assert type(mcm.env) is MDP, "El ambiente debe estar definido como un MDP"
except:
    raise Exception("El atributo env no está definido")
try:
    mcm.discount
    assert type(mcm.discount) is float, "La taza de descuento debe ser un número flotante"
except:
    raise Exception("La taza de descuento no esta definida como discount")
try:
    mcm.episodes
    assert type(mcm.episodes) is int, "La cantidad de episodios debe ser un entero"
except:
    raise Exception("El atributo episodes no está definido")
try:
    mcm.threshold
    assert type(mcm.threshold) is float, "El threshold debe ser un número flotante"
except:
    raise Exception("El atributo threshold no está definido")
try:
    mcm.values
    assert type(mcm.values) is dict, "Los valores deben estar definidos como un mapa"
except:
    raise Exception("El atributo values no está definido")
try:
    mcm.rewards
    assert type(mcm.rewards) is dict, "Las recompensas deben estar definidas como un mapa"
except:
    raise Exception("El atributo rewards no está definido")
try:
    mcm.policy
    assert type(mcm.policy) is dict, "Las políticas deben estar definidas como un mapa"
except:
    raise Exception("El atributo policy no está definido")


In [25]:
#Pruebas adicionales



#### 2. Generar episodios

Implemente la función `generate_episode` que, dado el agente, desde el estado actual, continuamente escoge una acción a realizar (aleatoriamente) y obtiene la recompensa correspondiente. Este proceso se repite hasta que se completa el episodio (se alcanza el objetivo, un estado final). La función debe retornar la secuencia de acciones como una lista (i.e., tuplas (estado, acción, recompensa) tomadas durante el episodio.

La generación del episodio comienza desde el estado inicial del ambiente y utiliza una política aleatoria para escoger la acción en cada estado visitado hasta llegar al estado final. Para cada iteración durante el episodio se acumula el número de muestras tomadas en la variable `samples` (inicializada en 0)

In [26]:
mcm = MCM(environment, discount=0.9, iterations=1000, threshold=0.00001)

try:
    mcm.generate_episode
    assert type(mcm.generate_episode()) is list, "El ambiente debe estar definido como un MDP"
except:
    raise Exception("La función generate_episode no está definida")



In [27]:
#Pruebas adicionales



#### 3. Evaluar la convergencias

Implemente la función de convergencia `check_convergence` que determina si los valores explorados convergen para las iteraciones realizadas. Esta función recibe los nuevos valores cálculados por parámetro y los compara con los valores cálculados en el último episodio (en el atributo `values`). Si la diferencia entre alguno de los valores nuevos y anteriores es superior al `threshold` definido para la clase, entonces actualizamos los valores de la clase y retornamos `False`. De lo contrario, la función retorna `True`.

In [28]:
mcm = MCM(environment, discount=0.9, iterations=1000, threshold=0.00001)

try:
    mcm.check_convergence
    assert type(mcm.check_convergence({'A': -10, 'B':3, 'C':-1, 'D':10, 'E':2.95})) is bool, "La convergencia debe retornar un booleano"
except:
    raise Exception("La función check_convergence no está definida")



In [29]:
#Pruebas adicionales



#### 4. Cálculo de las recompensas

Implemente la función `calculate_rewards` para un episodio (`episode`) dado por parámetro. Esta función debe tomar la secuencia de acciones (tuplas estado, accion, recompensa) del episodio y calcular la recompensa con descuento para cada uno de los estados. La función retorna un diccionario, con llave `estado` y valor lista de recompensas calculadas.

In [30]:
mcm = MCM(environment, discount=0.9, iterations=1000, threshold=0.00001)

try:
    mcm.calculate_rewards
    episodes = mcm.generate_episode()
    assert type(mcm.calculate_rewards(episodes)) is dict, "La convergencia debe retornar un mapa con las recompensas para cada estado"
except:
    raise Exception("La función calculate_rewards no está definida")



In [31]:
#Pruebas adicionales



#### 5.  Ejecución del agente

Finalmente, debe ejecutar el agente utilizando el ambiente de la para observar los resultados obtenidos. La ejecución del agente ejecuta el método de Monte Carlo por el número de episodios con los que se crea el agente. Dentro de cada episodio el agente observa las muestras de la ejecución desde el estado inicial hasta un estado final. Basado en las muestras el agente toma las recompensas obtenidas en cada iteración y actualiza las recompensas y los valores. luego, basado en los valores de cada estado, se calcula la política correspondiente. Esta función retorna el mapa de valores y el mápa de la política resultado luego de la ejecución de todos los episodios.


In [32]:
mcm = MCM(environment, discount=0.9, iterations=2, threshold=0.00001)

values, policy = mcm.run()
print(values)
print(policy)

{'D': 10.0, 'C': 9.0, 'B': 5.987102445, 'E': 8.1}
{'A': 'exit', 'C': 'down', 'E': 'up', 'B': 'right', 'D': 'exit'}
